# Profitable Apps on App Store and Google Play

The goal of this project is to find mobile apps that are profitable for our company that buildis free mobile apps. 

## Data

We have two datastes for Android ad iOS apps. We will base our subsequent analysis on these two datasets. The two datasets can be found here:

[Google Play Dataset](https://www.kaggle.com/lava18/google-play-store-apps/home)

[App Store Dataset](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home)





In [15]:
from csv import reader

### The Google Play data set ###
opened_file = open('googleplaystore.csv')
read_file = reader(opened_file)
android = list(read_file)
android_header = android[0]
android = android[1:]

### The App Store data set ###
opened_file = open('AppleStore.csv')
read_file = reader(opened_file)
ios = list(read_file)
ios_header = ios[0]
ios = ios[1:]

We will explore the datasets with a function called 'Explore Datasets'. We will print the column names and the first two rows of each dataset

In [16]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

print ('Android Dataset')
print ('\n')
print (android_header)
print ('\n')
explore_data(android,0,2,True)
print ('\n')

print ('iOS Dataset')
print ('\n')
print (ios_header)
print ('\n')
explore_data(ios,0,2,True)

Android Dataset


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


iOS Dataset


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '11

##  Cleaning Data

In the discussion forum, we can see that row 10472 is wrong and therefore we will delete it for subsequent analysis

In [17]:
print(android[10472])  # incorrect row
print('\n')
print(android_header)  # header
print('\n')
print(android[0])      # correct row

['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


In [18]:
print(len(android))
del android[10472]  # don't run this more than once
print(len(android))

10841
10840


After more exploration of the google play data set and the discussion forum, we will realize that many rows are duplicated. For example, instagram is shown four times. We will explore the number of duplicate apps and delete them

In [19]:
for app in android:
    name=app[0]
    if name == 'Instagram':
        print (app)

['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577446', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66577313', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']
['Instagram', 'SOCIAL', '4.5', '66509917', 'Varies with device', '1,000,000,000+', 'Free', '0', 'Teen', 'Social', 'July 31, 2018', 'Varies with device', 'Varies with device']


In [20]:
duplicate=[]
unique=[]

for app in android:
    name=app[0]
    if name in unique:
        duplicate.append(name)
    else:
        unique.append(name)
print ('number of duplicate apps:', len(duplicate))

number of duplicate apps: 1181


Based on the four Instagram rows, we can see that the difference lies on the number of reviews. We will delete the rows with less reviews, since these data were collected earlier. 

In [21]:
reviews_max={}
for app in android:
    name = app[0]
    n_reviews = float(app[3])
    if name in reviews_max and reviews_max[name]<n_reviews:
        reviews_max[name]=n_reviews
    if name not in reviews_max:
        reviews_max[name]=n_reviews

## Check Instagram value and length of the dictionary

print(reviews_max['Instagram'])

print('Expected length:', len(android) - 1181)
print('Actual length:', len(reviews_max))


66577446.0
Expected length: 9659
Actual length: 9659


Using the dictionary above we will create a cleaned android dataset

In [22]:
android_clean=[]
already_added=[]

for app in android:
    name=app[0]
    n_reviews = float(app[3])
    if n_reviews == reviews_max[name] and (name not in already_added):
        android_clean.append(app)
        already_added.append(name)

print(len(android_clean))
    

9659


We will check and remove all apps with non english names

In [23]:
def English_Name(string):
    for character in string:
        if ord(character)>127:
            return False
        
    return True
print(English_Name('Instagram'))
print(English_Name('s爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(English_Name('Docs To Go™ Free Office Suite'))
print(English_Name('Instachat 😜'))



True
False
False
False


We will modify the above function to label false only strings with thre or more non english characters, so the two later examples will be labeled as true

In [27]:
def is_eng(string):
    non_eng=0
    for character in string:
        if ord(character)>127:
            non_eng+=1        
    if non_eng>3:
        return False
    else:
        return True
print(is_eng('Instagram'))
print(is_eng('s爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_eng('Docs To Go™ Free Office Suite'))
print(is_eng('Instachat 😜'))

True
False
True
True


In [29]:
android_eng=[]
for app in android_clean:
    if is_eng(app[0]):
        android_eng.append(app)
ios_eng=[]
for app in ios:
    if is_eng(app[1]):
        ios_eng.append(app)
print(len(android_eng))

print(len(ios_eng))

9614
6183


Finally we will isolate only free apps for the analysis. Price is located at index 7 for android and at index 4 for ios

In [32]:
android_final=[]
for app in android_eng:
    if app[7]=='0':
        android_final.append(app)

ios_final=[]
for app in ios_eng:
    if app[4]=='0.0':
        ios_final.append(app)
        
print(len(android_final))

print(len(ios_final))

8864
3222


# Common Apps by Genre
## Part One
As we mentioned in the introduction, our aim is to determine the kinds of apps that are likely to attract more users because our revenue is highly influenced by the number of people using our apps.

To minimize risks and overhead, our validation strategy for an app idea is comprised of three steps:

Build a minimal Android version of the app, and add it to Google Play.
If the app has a good response from users, we then develop it further.
If the app is profitable after six months, we also build an iOS version of the app and add it to the App Store.
Because our end goal is to add the app on both the App Store and Google Play, we need to find app profiles that are successful on both markets. For instance, a profile that might work well for both markets might be a productivity app that makes use of gamification.

Let's begin the analysis by getting a sense of the most common genres for each market. For this, we'll build a frequency table for the prime_genre column of the App Store data set, and the Genres and Category columns of the Google Play data set.

## Part Two
We'll build two functions we can use to analyze the frequency tables:

One function to generate frequency tables that show percentages
Another function that we can use to display the percentages in a descending order

In [41]:
def freq_table(dataset, index):
    table = {}
    total = 0
    
    for row in dataset:
        total += 1
        value = row[index]
        if value in table:
            table[value] += 1
        else:
            table[value] = 1
    
    table_percentages = {}
    for key in table:
        percentage = (table[key] / total) * 100
        table_percentages[key] = percentage 
    
    return table_percentages

def display_table(dataset, index):
    table = freq_table(dataset, index)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])


In [42]:
display_table(ios_final, -5)

Games : 58.16263190564867
Entertainment : 7.883302296710118
Photo & Video : 4.9658597144630665
Education : 3.662321539416512
Social Networking : 3.2898820608317814
Shopping : 2.60707635009311
Utilities : 2.5139664804469275
Sports : 2.1415270018621975
Music : 2.0484171322160147
Health & Fitness : 2.0173805090006205
Productivity : 1.7380509000620732
Lifestyle : 1.5828677839851024
News : 1.3345747982619491
Travel : 1.2414649286157666
Finance : 1.1173184357541899
Weather : 0.8690254500310366
Food & Drink : 0.8069522036002483
Reference : 0.5586592178770949
Business : 0.5276225946617008
Book : 0.4345127250155183
Navigation : 0.186219739292365
Medical : 0.186219739292365
Catalogs : 0.12414649286157665


In [43]:

display_table(android_final, 1)

FAMILY : 18.907942238267147
GAME : 9.724729241877256
TOOLS : 8.461191335740072
BUSINESS : 4.591606498194946
LIFESTYLE : 3.9034296028880866
PRODUCTIVITY : 3.892148014440433
FINANCE : 3.7003610108303246
MEDICAL : 3.531137184115524
SPORTS : 3.395758122743682
PERSONALIZATION : 3.3167870036101084
COMMUNICATION : 3.2378158844765346
HEALTH_AND_FITNESS : 3.0798736462093865
PHOTOGRAPHY : 2.944494584837545
NEWS_AND_MAGAZINES : 2.7978339350180503
SOCIAL : 2.6624548736462095
TRAVEL_AND_LOCAL : 2.33528880866426
SHOPPING : 2.2450361010830324
BOOKS_AND_REFERENCE : 2.1435018050541514
DATING : 1.861462093862816
VIDEO_PLAYERS : 1.7937725631768955
MAPS_AND_NAVIGATION : 1.3989169675090252
FOOD_AND_DRINK : 1.2409747292418771
EDUCATION : 1.1620036101083033
ENTERTAINMENT : 0.9589350180505415
LIBRARIES_AND_DEMO : 0.9363718411552346
AUTO_AND_VEHICLES : 0.9250902527075812
HOUSE_AND_HOME : 0.8235559566787004
WEATHER : 0.8009927797833934
EVENTS : 0.7107400722021661
PARENTING : 0.6543321299638989
ART_AND_DESIGN : 

In [45]:
display_table(android_final, -4)


Tools : 8.449909747292418
Entertainment : 6.069494584837545
Education : 5.347472924187725
Business : 4.591606498194946
Productivity : 3.892148014440433
Lifestyle : 3.892148014440433
Finance : 3.7003610108303246
Medical : 3.531137184115524
Sports : 3.463447653429603
Personalization : 3.3167870036101084
Communication : 3.2378158844765346
Action : 3.1024368231046933
Health & Fitness : 3.0798736462093865
Photography : 2.944494584837545
News & Magazines : 2.7978339350180503
Social : 2.6624548736462095
Travel & Local : 2.3240072202166067
Shopping : 2.2450361010830324
Books & Reference : 2.1435018050541514
Simulation : 2.0419675090252705
Dating : 1.861462093862816
Arcade : 1.8501805054151623
Video Players & Editors : 1.7712093862815883
Casual : 1.7599277978339352
Maps & Navigation : 1.3989169675090252
Food & Drink : 1.2409747292418771
Puzzle : 1.128158844765343
Racing : 0.9927797833935018
Role Playing : 0.9363718411552346
Libraries & Demo : 0.9363718411552346
Auto & Vehicles : 0.9250902527075

## Most Popular Apps by Genre on the App Store
One way to find out what genres are the most popular (have the most users) is to calculate the average number of installs for each app genre. For the Google Play data set, we can find this information in the Installs column, but for the App Store data set this information is missing. As a workaround, we'll take the total number of user ratings as a proxy, which we can find in the rating_count_tot app.

Below, we calculate the average number of user ratings per app genre on the App Store:

In [47]:
genres_ios = freq_table(ios_final, -5)

for genre in genres_ios:
    total=0
    len_genre=0
    for app in ios_final:
        genre_app=app[-5]
        if genre_app == genre:
            value = float(app[5])
            total+=value
            len_genre+=1
    avg = total/len_genre
    print (genre,':',avg)


Navigation : 86090.33333333333
Social Networking : 71548.34905660378
Games : 22788.6696905016
Medical : 612.0
Shopping : 26919.690476190477
Book : 39758.5
Food & Drink : 33333.92307692308
Utilities : 18684.456790123455
Health & Fitness : 23298.015384615384
Productivity : 21028.410714285714
Sports : 23008.898550724636
Catalogs : 4004.0
Entertainment : 14029.830708661417
Travel : 28243.8
Lifestyle : 16485.764705882353
Music : 57326.530303030304
Education : 7003.983050847458
News : 21248.023255813954
Photo & Video : 28441.54375
Reference : 74942.11111111111
Weather : 52279.892857142855
Finance : 31467.944444444445
Business : 7491.117647058823


On average, navigation apps have the highest number of user reviews, but this figure is heavily influenced by Waze and Google Maps, which have close to half a million user reviews together:

## Most Popular Apps by Genre on Google Play
For the Google Play market, we actually have data about the number of installs, so we should be able to get a clearer picture about genre popularity. However, the install numbers don't seem precise enough — we can see that most values are open-ended (100+, 1,000+, 5,000+, etc.):

In [51]:
categories_android = freq_table(android_final,1)

In [52]:
for category in categories_android:
    total=0
    len_category=0
    for app in android_final:
        category_app=app[1]
        if category_app==category:
            installs=app[5]
            installs = installs.replace('+','')
            installs = installs.replace(',','')
            installs = float(installs)
            total+=installs
            len_category+=1
    avg = total/len_category
    print(category,':',avg)

HOUSE_AND_HOME : 1331540.5616438356
NEWS_AND_MAGAZINES : 9549178.467741935
BUSINESS : 1712290.1474201474
SOCIAL : 23253652.127118643
SHOPPING : 7036877.311557789
ENTERTAINMENT : 11640705.88235294
COMICS : 817657.2727272727
PERSONALIZATION : 5201482.6122448975
LIBRARIES_AND_DEMO : 638503.734939759
BEAUTY : 513151.88679245283
BOOKS_AND_REFERENCE : 8767811.894736841
PHOTOGRAPHY : 17840110.40229885
LIFESTYLE : 1437816.2687861272
FAMILY : 3695641.8198090694
MEDICAL : 120550.61980830671
EDUCATION : 1833495.145631068
VIDEO_PLAYERS : 24727872.452830188
TOOLS : 10801391.298666667
PRODUCTIVITY : 16787331.344927534
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
HEALTH_AND_FITNESS : 4188821.9853479853
TRAVEL_AND_LOCAL : 13984077.710144928
WEATHER : 5074486.197183099
COMMUNICATION : 38456119.167247385
AUTO_AND_VEHICLES : 647317.8170731707
GAME : 15588015.603248259
PARENTING : 542603.6206896552
DATING : 854028.8303030303
SPORTS : 3638640.1428571427
EVENTS : 253542.2222222222

On average, communication apps have the most installs: 38,456,119. This number is heavily skewed up by a few apps that have over one billion installs (WhatsApp, Facebook Messenger, Skype, Google Chrome, Gmail, and Hangouts), and a few others with over 100 and 500 million installs.

The books and reference genre looks fairly popular as well, with an average number of installs of 8,767,811. It's interesting to explore this in more depth, since we found this genre has some potential to work well on the App Store, and our aim is to recommend an app genre that shows potential for being profitable on both the App Store and Google Play.

# Conclusions
In this project, we analyzed data about the App Store and Google Play mobile apps with the goal of recommending an app profile that can be profitable for both markets.

We concluded that taking a popular book (perhaps a more recent book) and turning it into an app could be profitable for both the Google Play and the App Store markets. The markets are already full of libraries, so we need to add some special features besides the raw version of the book. This might include daily quotes from the book, an audio version of the book, quizzes on the book, a forum where people can discuss the book, etc.